In [11]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim

In [12]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307, ), (0.3081, ))])

train_data = datasets.MNIST(root="./dataset", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="./dataset", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [13]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)

model = Net()

criterion = torch.nn.CrossEntropyLoss()
optim = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [14]:
def train(epoch):
    running_loss = 0.0
    for batch_inx, data in enumerate(train_loader, 0):
        input, target = data
        optim.zero_grad()

        outputs = model(input)
        loss = criterion(outputs, target)
        loss.backward()
        optim.step()

        running_loss += loss.item()
        if batch_inx % 300 == 299:
            print("[%d, %5d] loss: %.3f" %(epoch + 1, batch_inx + 1, running_loss / 300))
            running_loss = 0

def test():
    correct = 0
    total = 0

    # python中with语句的介绍 https://blog.csdn.net/ego_bai/article/details/80873242
    with torch.no_grad():  # 关于torch.no_grad()的介绍 https://blog.csdn.net/Answer3664/article/details/99460175
        for data in test_loader:
            images, label = data
            outputs = model(images)
            _, predict = torch.max(outputs.data, dim=1)  # torch.max的返回值有两个，第一个是每一行的最大值是多少，第二个是每一行最大值的下标(索引)是多少。
            total += label.size(0)
            correct += (predict == label).sum().item()
    print("Accuracy on test set: %d %%" %(100 * correct / total))

In [15]:
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 2.233
[1,   600] loss: 1.015
[1,   900] loss: 0.427
Accuracy on test set: 90 %
[2,   300] loss: 0.320
[2,   600] loss: 0.270
[2,   900] loss: 0.225
Accuracy on test set: 93 %
[3,   300] loss: 0.192
[3,   600] loss: 0.171
[3,   900] loss: 0.149
Accuracy on test set: 95 %
[4,   300] loss: 0.133
[4,   600] loss: 0.124
[4,   900] loss: 0.115
Accuracy on test set: 95 %
[5,   300] loss: 0.099
[5,   600] loss: 0.094
[5,   900] loss: 0.097
Accuracy on test set: 96 %
[6,   300] loss: 0.080
[6,   600] loss: 0.078
[6,   900] loss: 0.074
Accuracy on test set: 97 %
[7,   300] loss: 0.062
[7,   600] loss: 0.065
[7,   900] loss: 0.066
Accuracy on test set: 97 %
[8,   300] loss: 0.052
[8,   600] loss: 0.055
[8,   900] loss: 0.050
Accuracy on test set: 97 %
[9,   300] loss: 0.039
[9,   600] loss: 0.046
[9,   900] loss: 0.046
Accuracy on test set: 97 %
[10,   300] loss: 0.033
[10,   600] loss: 0.035
[10,   900] loss: 0.038
Accuracy on test set: 97 %
